## 문제 1. 
- 다음은 Used car 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 중고차의 판매 가격을 예측하고 해당 예측 결과를 csv 파일로 제출하시오.
- 결과 제출 양식 : 제출한 예측값의 rmse 평가지표 값을 통해 영역별 배점에 따라 최종 점수가 반영될 예정

In [1]:
import pandas as pd 
pd.DataFrame({
    "ID"  : [1, 2, 3],
    "y_pred" : [12500, 16500, 11000]
})

,ID,y_pred
0,1,12500
1,2,16500
2,3,11000


In [2]:
import pandas as pd 
X_test = pd.read_csv("data/5회/504_x_test.csv")
X_train = pd.read_csv("data/5회/504_x_train.csv")
y_train = pd.read_csv("data/5회/504_y_train.csv")

In [3]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      7468 non-null   int64
 1   price   7468 non-null   int64
dtypes: int64(2)
memory usage: 116.8 KB


In [4]:
import pandas as pd 
pd.DataFrame({
    "변수" : list(X_train.columns) + ["price"], 
    "설명" : [
        "중고차 ID 번호", 
        "차량 모델명", 
        "차량 등록 연도", 
        "변속기 종류", 
        "주행 거리", 
        "엔진 연료 종류",
        "도로세", 
        "갤런당 마일(miles per gallon), 연비", 
        "엔진 크기",   
        "가격(파운드)"
    ]
})

,변수,설명
0,id,중고차 ID 번호
1,model,차량 모델명
2,year,차량 등록 연도
3,transmission,변속기 종류
4,mileage,주행 거리
5,fuelType,엔진 연료 종류
6,tax,도로세
7,mpg,"갤런당 마일(miles per gallon), 연비"
8,engineSize,엔진 크기
9,price,가격(파운드)


## 데이터 확인 

In [5]:
X_train.head(1)

,id,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,1,A1,2019,Automatic,3500,Petrol,145,40.9,2.0


In [6]:
X_test.head(1)

,id,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,7469,A4,2020,Semi-Auto,4000,Diesel,145,54.3,3.0


In [7]:
y_train.head(1)

,id,price
0,1,21350


## 결측치 확인

In [8]:
X_train.isnull().sum()

id              0
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [9]:
X_test.isnull().sum()

id              0
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

## 컬럼 확인

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7468 non-null   int64  
 1   model         7468 non-null   object 
 2   year          7468 non-null   int64  
 3   transmission  7468 non-null   object 
 4   mileage       7468 non-null   int64  
 5   fuelType      7468 non-null   object 
 6   tax           7468 non-null   int64  
 7   mpg           7468 non-null   float64
 8   engineSize    7468 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 525.2+ KB


In [11]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            3200 non-null   int64  
 1   model         3200 non-null   object 
 2   year          3200 non-null   int64  
 3   transmission  3200 non-null   object 
 4   mileage       3200 non-null   int64  
 5   fuelType      3200 non-null   object 
 6   tax           3200 non-null   int64  
 7   mpg           3200 non-null   float64
 8   engineSize    3200 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 225.1+ KB


In [12]:
X_train['fuelType'].value_counts()

fuelType
Diesel    3966
Petrol    3484
Hybrid      18
Name: count, dtype: int64

In [13]:
X_train['model'].value_counts()

model
 A3     1361
 Q3     1010
 A4      968
 A1      952
 Q5      613
 A5      609
 Q2      576
 A6      522
 Q7      257
 TT      233
 A7       86
 A8       78
 Q8       50
 RS6      24
 RS4      24
 RS5      23
 RS3      21
 R8       17
 SQ5      13
 S4        9
 S3        9
 SQ7       4
 S8        4
 S5        3
 A2        1
 RS7       1
Name: count, dtype: int64

- fuelType 기반으로 분리한다. 데이터셋 분리 한다.
- model은 제거한다. 

In [14]:
X_train = X_train.drop('model', axis = 1)
X_test = X_test.drop('model', axis = 1)
X_train.shape, X_test.shape

((7468, 8), (3200, 8))

In [15]:
X_train.describe()

,id,year,mileage,tax,mpg,engineSize
count,7468.000000,7468.000000,7468.000000,7468.000000,7468.000000,7468.000000
mean,3734.500000,2017.084762,24978.337306,125.513524,50.980704,1.923741
std,2155.970238,2.143985,23408.243740,66.791388,12.781446,0.594344
min,1.000000,1998.000000,1.000000,0.000000,19.300000,0.000000
25%,1867.750000,2016.000000,6000.000000,125.000000,40.900000,1.500000
50%,3734.500000,2017.000000,19276.500000,145.000000,50.400000,2.000000
75%,5601.250000,2019.000000,36722.750000,145.000000,58.900000,2.000000
max,7468.000000,2020.000000,168017.000000,580.000000,188.300000,6.300000


In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7468 non-null   int64  
 1   year          7468 non-null   int64  
 2   transmission  7468 non-null   object 
 3   mileage       7468 non-null   int64  
 4   fuelType      7468 non-null   object 
 5   tax           7468 non-null   int64  
 6   mpg           7468 non-null   float64
 7   engineSize    7468 non-null   float64
dtypes: float64(2), int64(4), object(2)
memory usage: 466.9+ KB


- MinMaxScaler : mileage, tax, mpg
- OneHotEncoder : transmission, fuelType 

## ID 제거

In [17]:
X_train_id = X_train.pop('id')
X_test_id = X_test.pop('id')

X_train.shape, X_test.shape

((7468, 7), (3200, 7))

## 컬럼 추출하기

In [18]:
import numpy as np
cat_cols = list(X_train.select_dtypes(exclude = np.number).columns)
num_cols = list(X_train.select_dtypes(include = np.number).columns)

print(cat_cols)
print(num_cols)

['transmission', 'fuelType']
['year', 'mileage', 'tax', 'mpg', 'engineSize']


In [19]:
num_cols.remove('year')
num_cols.remove('engineSize')
num_cols

['mileage', 'tax', 'mpg']

## 데이터셋 분리

In [20]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train['price'], 
    stratify = X_train['fuelType'], 
    test_size = 0.3, 
    random_state=42
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((5227, 7), (2241, 7), (5227,), (2241,))

## 모델 학습

In [22]:
import sklearn
# sklearn.metrics.SCORERS.keys()
from sklearn.metrics._scorer import _SCORERS
_SCORERS

{'explained_variance': make_scorer(explained_variance_score),
 'r2': make_scorer(r2_score),
 'max_error': make_scorer(max_error, greater_is_better=False),
 'matthews_corrcoef': make_scorer(matthews_corrcoef),
 'neg_median_absolute_error': make_scorer(median_absolute_error, greater_is_better=False),
 'neg_mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
 'neg_mean_absolute_percentage_error': make_scorer(mean_absolute_percentage_error, greater_is_better=False),
 'neg_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False),
 'neg_mean_squared_log_error': make_scorer(mean_squared_log_error, greater_is_better=False),
 'neg_root_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False, squared=False),
 'neg_mean_poisson_deviance': make_scorer(mean_poisson_deviance, greater_is_better=False),
 'neg_mean_gamma_deviance': make_scorer(mean_gamma_deviance, greater_is_better=False),
 'accuracy': make_scorer(accuracy_score),
 

In [26]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

# utils
from scipy.stats import loguniform

def cal_rmse(y_tr, pred):
    return np.sqrt(mean_squared_error(y_tr, pred))

param_grid = {
    "reg__learning_rate" : loguniform(0.001, 0.1), 
    "reg__n_estimators" : np.arange(50, 200), 
    "reg__max_depth" : np.arange(3, 30, 3), 
    "reg__num_leaves" : np.arange(20, 50), 
    "reg__min_split_gain" : np.arange(0, 1.1, 0.1), 
    "reg__subsample" : np.arange(0.6, 1.0, 0.1)
}

column_transformer = ColumnTransformer([
    ("scaler", MinMaxScaler(), num_cols), 
    ("ohc_encoder", OneHotEncoder(), cat_cols)
], remainder = "passthrough")

pipeline = Pipeline([
    ("preprocessor", column_transformer), 
    ("reg", LGBMRegressor(random_state = 42))
])

random_search = RandomizedSearchCV(
    estimator = pipeline, 
    param_distributions = param_grid, 
    n_iter = 10, 
    # scoring="neg_mean_squared_error", 
    scoring = make_scorer(cal_rmse, greater_is_better = False),
    cv = 5, 
    verbose=1, 
    n_jobs=1
)

random_search.fit(X_tr, y_tr)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000456 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 395
[LightGBM] [Info] Number of data points in the train set: 4181, number of used features: 10
[LightGBM] [Info] Start training from score 22518.030854
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000137 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 395
[LightGBM] [Info] Number of data points in the train set: 4181, number of used features: 10
[LightGBM] [Info] Start training from score 22724.556326
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000151 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total B

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('scaler',
                                                                               MinMaxScaler(),
                                                                               ['mileage',
                                                                                'tax',
                                                                                'mpg']),
                                                                              ('ohc_encoder',
                                                                               OneHotEncoder(),
                                                                               ['transmission',
                                                                                'fuelType'])])),
                                             ('reg',
                                              LGBMRegressor(random_state=42))]),
                   n_jobs=1,
                   param_distributions={'reg__learning_rate': <scipy.stats._distn_inf...
       167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
       180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192,
       193, 194, 195, 196, 197, 198, 199]),
                                        'reg__num_leaves': array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                                        'reg__subsample': array([0.6, 0.7, 0.8, 0.9])},
                   scoring=make_scorer(cal_rmse, greater_is_better=False),
                   verbose=1)

In [ ]:
def get_scores(model, X_tr, X_val, y_tr, y_val):
    y_tr_pred = model.predict(X_tr)
    y_val_pred = model.predict(X_val)
    tr_score = cal_rmse(y_tr, y_tr_pred)
    val_score = cal_rmse(y_val, y_val_pred)
    return f'train: {round(tr_score, 4)}, valid: {round(val_score, 4)}'

get_scores(random_search, X_tr, X_val, y_tr, y_val)

In [27]:
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best_Params:", best_params)
print("Best_Scores:", best_score)

Best_Params: {'reg__learning_rate': 0.03653848012627035, 'reg__max_depth': 24, 'reg__min_split_gain': 0.6000000000000001, 'reg__n_estimators': 113, 'reg__num_leaves': 38, 'reg__subsample': 0.8999999999999999}
Best_Scores: -3259.760793554663


## 결과 제출

In [28]:
final_preds = random_search.predict(X_test)
final_preds

array([39101.35975967, 16939.71370835, 33753.33987638, ...,
       18110.15305548, 24539.184126  , 17646.89091796])

In [29]:
result = pd.DataFrame({"id" : X_test_id, "price": np.round(final_preds, 0)})
result

,id,price
0,7469,39101.0
1,7470,16940.0
2,7471,33753.0
3,7472,17680.0
4,7473,14012.0
...,...,...
3195,10664,18318.0
3196,10665,11747.0
3197,10666,18110.0
3198,10667,24539.0
